# 1. Mô tả bộ dữ liệu
### Nguồn dữ liệu:
Bộ dữ liệu Auto MPG được công bố trên UCI Machine Learning Repository, mô tả các thông số kỹ thuật và mức tiêu thụ nhiên liệu của các mẫu xe ô tô được sản xuất tại Mỹ từ năm 1970–1982.
### Mục tiêu bài toán:
Dự đoán mức tiêu thụ nhiên liệu (MPG – Miles per Gallon) của xe dựa trên các đặc trưng kỹ thuật như dung tích xi-lanh, công suất, trọng lượng, năm sản xuất.

→ Đây là bài toán hồi quy (regression), trong đó biến mục tiêu là MPG.

### Quy mô dữ liệu:
398 mẫu, 8 thuộc tính. Một cột (Horsepower) có giá trị thiếu chiếm khoảng 1.5% tổng dữ liệu.
### Các thuộc tính chính:
| Thuộc tính   | Mô tả                                         | Kiểu dữ liệu               |
| ------------ | --------------------------------------------- | -------------------------- |
| MPG          | Mức tiêu thụ nhiên liệu (Miles per Gallon)    | Numeric (Target)           |
| Cylinders    | Số xi-lanh của động cơ                        | Numeric (Discrete)         |
| Displacement | Dung tích xi-lanh (inch³)                     | Numeric                    |
| Horsepower   | Công suất động cơ (HP)                        | Numeric (có giá trị thiếu) |
| Weight       | Khối lượng xe (pound)                         | Numeric                    |
| Acceleration | Thời gian tăng tốc từ 0–60 mph (giây)         | Numeric                    |
| Model Year   | Năm sản xuất (1970–1982)                      | Numeric (Discrete)         |
| Origin       | Nguồn gốc xe (1: Mỹ, 2: Châu Âu, 3: Nhật Bản) | Categorical (Mã hóa số)    |


### Đặc điểm dữ liệu:

- Dữ liệu có 5 cột số thực và 3 cột số nguyên.

- Cột Horsepower chứa một số giá trị thiếu (~1.5%), cần được xử lý.

- Một số thuộc tính (Cylinders, Origin, Model Year) mang tính phân loại rời rạc.

- Các giá trị có thang đo khác nhau → cần chuẩn hóa hoặc scale trước khi huấn luyện mô hình.

### Chất lượng và xử lý dữ liệu:

- Điền giá trị thiếu cho Horsepower bằng trung vị hoặc giá trị dự đoán.

- Có thể mã hóa cột Origin thành biến giả (One-Hot Encoding).

- Chuẩn hóa dữ liệu số bằng StandardScaler hoặc MinMaxScaler để tối ưu hiệu suất mô hình.

### Ứng dụng và phạm vi sử dụng:

- Dự đoán mức tiêu thụ nhiên liệu để đánh giá hiệu quả năng lượng của xe.

- Hỗ trợ phân tích kỹ thuật động cơ và thiết kế xe tiết kiệm nhiên liệu.

- Là bộ dữ liệu kinh điển phục vụ học tập và nghiên cứu trong các mô hình hồi quy:

  - Linear Regression

  - Random Forest Regressor

  - Gradient Boosting

  - Support Vector Regressor (SVR)

# 2. Quy trình chia dữ liệu và huấn luyện mô hình Linear Regression

## 2.1 Tải và đọc dữ liệu

In [ ]:
import pandas as pd

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

print("  PHÂN TÍCH CÁC NHÃN/GIÁ TRỊ ĐỘC NHẤT TRONG DỮ LIỆU  ")

for col in raw_dataset.columns:
    print(f"\n--- Cột: {col} (Dtype: {raw_dataset[col].dtype}) ---")

    if raw_dataset[col].dtype == 'object':
        print("Các nhãn/giá trị độc nhất và tần suất:")
        print(raw_dataset[col].value_counts(dropna=False).head(10))
        print(f"Tổng số giá trị độc nhất: {raw_dataset[col].nunique()}")

    elif raw_dataset[col].dtype == 'int64':
        if raw_dataset[col].nunique() <= 10:
            print("Các giá trị/nhãn số nguyên và tần suất:")
            print(raw_dataset[col].value_counts(dropna=False))
        else:
            print(f"Số lượng giá trị độc nhất: {raw_dataset[col].nunique()}. "
                  f"Giá trị nhỏ nhất: {raw_dataset[col].min()}, Giá trị lớn nhất: {raw_dataset[col].max()}.")

    elif raw_dataset[col].dtype == 'float64':
        print(f"Số lượng giá trị độc nhất: {raw_dataset[col].nunique()}. "
              f"Giá trị nhỏ nhất: {raw_dataset[col].min()}, Giá trị lớn nhất: {raw_dataset[col].max()}.")
        print(f"Tỷ lệ giá trị thiếu (NaN): {raw_dataset[col].isnull().sum() / len(raw_dataset) * 100:.2f}%")

print("THÔNG TIN TỔNG QUAN DỮ LIỆU:")
print(raw_dataset.info())
print("\n5 dòng đầu tiên:")
print(raw_dataset.head())


  PHÂN TÍCH CÁC NHÃN/GIÁ TRỊ ĐỘC NHẤT TRONG DỮ LIỆU  

--- Cột: MPG (Dtype: float64) ---
Số lượng giá trị độc nhất: 129. Giá trị nhỏ nhất: 9.0, Giá trị lớn nhất: 46.6.
Tỷ lệ giá trị thiếu (NaN): 0.00%

--- Cột: Cylinders (Dtype: int64) ---
Các giá trị/nhãn số nguyên và tần suất:
Cylinders
4    204
8    103
6     84
3      4
5      3
Name: count, dtype: int64

--- Cột: Displacement (Dtype: float64) ---
Số lượng giá trị độc nhất: 82. Giá trị nhỏ nhất: 68.0, Giá trị lớn nhất: 455.0.
Tỷ lệ giá trị thiếu (NaN): 0.00%

--- Cột: Horsepower (Dtype: float64) ---
Số lượng giá trị độc nhất: 93. Giá trị nhỏ nhất: 46.0, Giá trị lớn nhất: 230.0.
Tỷ lệ giá trị thiếu (NaN): 1.51%

--- Cột: Weight (Dtype: float64) ---
Số lượng giá trị độc nhất: 351. Giá trị nhỏ nhất: 1613.0, Giá trị lớn nhất: 5140.0.
Tỷ lệ giá trị thiếu (NaN): 0.00%

--- Cột: Acceleration (Dtype: float64) ---
Số lượng giá trị độc nhất: 95. Giá trị nhỏ nhất: 8.0, Giá trị lớn nhất: 24.8.
Tỷ lệ giá trị thiếu (NaN): 0.00%

--- Cột: Model Y

## 2.2 Kiểm tra thông tin và xử lý dữ liệu

In [ ]:
print("Thông tin tổng quan:")
raw_dataset.info()

print("\nGiá trị thiếu:")
print(raw_dataset.isnull().sum())

# Điền giá trị thiếu cho cột 'Horsepower' (nếu có)
if 'Horsepower' in raw_dataset.columns:
    raw_dataset['Horsepower'] = raw_dataset['Horsepower'].fillna(raw_dataset['Horsepower'].median())

from sklearn.preprocessing import LabelEncoder

if 'Origin' in raw_dataset.columns:
    le = LabelEncoder()
    raw_dataset['Origin'] = le.fit_transform(raw_dataset['Origin'].astype(str))

# Kiểm tra sau khi xử lý
print("\nSau khi tiền xử lý:")
print(raw_dataset.head())

# Đảm bảo không còn giá trị thiếu
raw_dataset = raw_dataset.dropna()

print("\nSau khi loại bỏ giá trị thiếu còn lại:")
print(raw_dataset.isnull().sum())


Thông tin tổng quan:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           398 non-null    float64
 1   Cylinders     398 non-null    int64  
 2   Displacement  398 non-null    float64
 3   Horsepower    392 non-null    float64
 4   Weight        398 non-null    float64
 5   Acceleration  398 non-null    float64
 6   Model Year    398 non-null    int64  
 7   Origin        398 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 25.0 KB

Giá trị thiếu:
MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

Sau khi tiền xử lý:
    MPG  Cylinders  Displacement  Horsepower  Weight  Acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0        

## 2.3 Chia dữ liệu

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Tách biến đầu vào (X) và biến mục tiêu (y)
X = raw_dataset.drop(columns=['MPG'])
y = raw_dataset['MPG']

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Kích thước tập huấn luyện:", X_train.shape)
print("Kích thước tập kiểm tra:", X_test.shape)


Kích thước tập huấn luyện: (278, 7)
Kích thước tập kiểm tra: (120, 7)


## 2.4 Huấn luyện mô hình Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Khởi tạo mô hình
lr_model = LinearRegression()

# Huấn luyện mô hình
lr_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = lr_model.predict(X_test)

## 2.5 Đánh giá mô hình

In [ ]:
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("============================================")
print("   KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH LINEAR REGRESSION")
print("============================================")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.3f}")

# (Tùy chọn) In hệ số hồi quy
print("\nHệ số hồi quy (Coefficients):")
for name, coef in zip(raw_dataset.drop(columns=['MPG']).columns, lr_model.coef_):
    print(f"{name}: {coef:.3f}")
print(f"\nHệ số chặn (Intercept): {lr_model.intercept_:.3f}")

   KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH LINEAR REGRESSION
Mean Squared Error (MSE): 9.27
R² Score: 0.838

Hệ số hồi quy (Coefficients):
Cylinders: 0.041
Displacement: 0.736
Horsepower: -0.173
Weight: -5.392
Acceleration: 0.353
Model Year: 2.793
Origin: 1.196

Hệ số chặn (Intercept): 23.447
